# RNN with time series

In [15]:
import pandas as pd
import numpy as np
from datetime import date, datetime
from dateutil import relativedelta
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense
import plotly.offline as pyoff
import plotly.graph_objects as go

df = pd.read_csv('raw_sales.csv') 
df.head()

,datesold,postcode,price,propertyType,bedrooms
0,2007-02-07 00:00:00,2607,525000,house,4
1,2007-02-27 00:00:00,2906,290000,house,3
2,2007-03-07 00:00:00,2905,328000,house,3
3,2007-03-09 00:00:00,2905,380000,house,4
4,2007-03-21 00:00:00,2906,310000,house,3


In [16]:
# Cambio de fechas en columna 'datesold', dejando sólo año y mes. 
for i in range(len(df)):
    df['datesold'][i] = df['datesold'][i].split()[0][:-3]

# Eliminar columnas que no usaremos y cambiar de nombre 'datesold' y 'price' por 'mes' y 'ventas'.
df = df.drop(['postcode', 'propertyType', 'bedrooms'], axis=1)
df = df.rename(columns={'datesold': 'mes', 'price': 'ventas'})

# Agrupar meses, sumando las ventas totales del mes.
df = df.groupby('mes')['ventas'].agg('sum')
df = df.reset_index()
df.head()

C:\Users\JOSELU~1\AppData\Local\Temp/ipykernel_660/4048521769.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,mes,ventas
0,2007-02,815000
1,2007-03,1018000
2,2007-04,2394000
3,2007-05,679000
4,2007-06,3122000


In [14]:
plot_data = [
go.Scatter(
x=df['mes'],
y=df['ventas']
)
]
plot_layout = go.Layout(
title='Ventas mensuales'
)
fig = go.Figure(data=plot_data, layout=plot_layout) 
pyoff.iplot(fig)

In [17]:
df_diff = df.copy()
df_diff['prev_ventas'] = df_diff['ventas'].shift(1)
df_diff = df_diff.dropna()
df_diff['diff'] = (df_diff['ventas'] - df_diff['prev_ventas'])
df_diff

,mes,ventas,prev_ventas,diff
1,2007-03,1018000,815000.0,203000.0
2,2007-04,2394000,1018000.0,1376000.0
3,2007-05,679000,2394000.0,-1715000.0
4,2007-06,3122000,679000.0,2443000.0
5,2007-07,11249500,3122000.0,8127500.0
...,...,...,...,...
145,2019-03,153793350,127458810.0,26334540.0
146,2019-04,138929019,153793350.0,-14864331.0
147,2019-05,141128050,138929019.0,2199031.0
148,2019-06,88612700,141128050.0,-52515350.0


In [18]:
plot_data = [
go.Scatter(
x=df_diff['mes'],
y=df_diff['ventas']
)
]
plot_layout = go.Layout(
title='Ventas mensuales'
)
fig = go.Figure(data=plot_data, layout=plot_layout) 
pyoff.iplot(fig)

In [20]:
df_supervised = df_diff.drop(['prev_ventas'],axis=1)

for inc in range(1,13): 
    field_name = 'lag_' + str(inc) 
    df_supervised[field_name] = df_supervised['diff'].shift(inc)

df_supervised = df_supervised.dropna().reset_index(drop=True) 
df_supervised

,mes,ventas,diff,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12
0,2008-03,3381000,-9896000.0,6215000.0,-4569950.0,-1509550.0,3169500.0,-1165500.0,-491500.0,379500.0,8127500.0,2443000.0,-1715000.0,1376000.0,203000.0
1,2008-04,11752500,8371500.0,-9896000.0,6215000.0,-4569950.0,-1509550.0,3169500.0,-1165500.0,-491500.0,379500.0,8127500.0,2443000.0,-1715000.0,1376000.0
2,2008-05,9925500,-1827000.0,8371500.0,-9896000.0,6215000.0,-4569950.0,-1509550.0,3169500.0,-1165500.0,-491500.0,379500.0,8127500.0,2443000.0,-1715000.0
3,2008-06,18329150,8403650.0,-1827000.0,8371500.0,-9896000.0,6215000.0,-4569950.0,-1509550.0,3169500.0,-1165500.0,-491500.0,379500.0,8127500.0,2443000.0
4,2008-07,17198500,-1130650.0,8403650.0,-1827000.0,8371500.0,-9896000.0,6215000.0,-4569950.0,-1509550.0,3169500.0,-1165500.0,-491500.0,379500.0,8127500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,2019-03,153793350,26334540.0,44833610.0,-65110252.0,-60997259.0,20533974.0,-2150360.0,-445179.0,3628226.0,-34771600.0,-74131876.0,57985071.0,-66859870.0,31581636.0
133,2019-04,138929019,-14864331.0,26334540.0,44833610.0,-65110252.0,-60997259.0,20533974.0,-2150360.0,-445179.0,3628226.0,-34771600.0,-74131876.0,57985071.0,-66859870.0
134,2019-05,141128050,2199031.0,-14864331.0,26334540.0,44833610.0,-65110252.0,-60997259.0,20533974.0,-2150360.0,-445179.0,3628226.0,-34771600.0,-74131876.0,57985071.0
135,2019-06,88612700,-52515350.0,2199031.0,-14864331.0,26334540.0,44833610.0,-65110252.0,-60997259.0,20533974.0,-2150360.0,-445179.0,3628226.0,-34771600.0,-74131876.0


In [21]:
df_model = df_supervised.drop(['ventas','mes'],axis=1)
train_set, test_set = df_model[0:-12].values, df_model[-12:].values
df_model.head()

,diff,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12
0,-9896000.0,6215000.0,-4569950.0,-1509550.0,3169500.0,-1165500.0,-491500.0,379500.0,8127500.0,2443000.0,-1715000.0,1376000.0,203000.0
1,8371500.0,-9896000.0,6215000.0,-4569950.0,-1509550.0,3169500.0,-1165500.0,-491500.0,379500.0,8127500.0,2443000.0,-1715000.0,1376000.0
2,-1827000.0,8371500.0,-9896000.0,6215000.0,-4569950.0,-1509550.0,3169500.0,-1165500.0,-491500.0,379500.0,8127500.0,2443000.0,-1715000.0
3,8403650.0,-1827000.0,8371500.0,-9896000.0,6215000.0,-4569950.0,-1509550.0,3169500.0,-1165500.0,-491500.0,379500.0,8127500.0,2443000.0
4,-1130650.0,8403650.0,-1827000.0,8371500.0,-9896000.0,6215000.0,-4569950.0,-1509550.0,3169500.0,-1165500.0,-491500.0,379500.0,8127500.0


In [22]:
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(train_set)

train_set = train_set.reshape(train_set.shape[0], train_set.shape[1])
train_set_scaled = scaler.transform(train_set)

test_set = test_set.reshape(test_set.shape[0], test_set.shape[1])
test_set_scaled = scaler.transform(test_set)

In [23]:
X_train, y_train = train_set_scaled[:, 1:], train_set_scaled[:, 0:1]
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test, y_test = test_set_scaled[:, 1:], test_set_scaled[:, 0:1]
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])


# Ajustando el modelo
model = Sequential()
model.add(LSTM(4, batch_input_shape=(1, X_train.shape[1], X_train.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=100, batch_size=1, verbose=1, shuffle=False)

Epoch 1/100
125/125 [==============================] - 2s 3ms/step - loss: 0.0879
Epoch 2/100
125/125 [==============================] - 0s 3ms/step - loss: 0.0759
Epoch 3/100
125/125 [==============================] - 0s 3ms/step - loss: 0.0666
Epoch 4/100
125/125 [==============================] - 0s 3ms/step - loss: 0.0580
Epoch 5/100
125/125 [==============================] - 0s 3ms/step - loss: 0.0499
Epoch 6/100
125/125 [==============================] - 0s 3ms/step - loss: 0.0428
Epoch 7/100
125/125 [==============================] - 0s 2ms/step - loss: 0.0370
Epoch 8/100
125/125 [==============================] - 0s 3ms/step - loss: 0.0325
Epoch 9/100
125/125 [==============================] - 0s 3ms/step - loss: 0.0291
Epoch 10/100
125/125 [==============================] - 0s 4ms/step - loss: 0.0264
Epoch 11/100
125/125 [==============================] - 1s 4ms/step - loss: 0.0243
Epoch 12/100
125/125 [==============================] - 0s 3ms/step - loss: 0.0225
Epoch 13/100


In [25]:
# Predicción del modelo.
y_pred = model.predict(X_test,batch_size=1)

# Transformación inversa del escalamiento.
y_pred = y_pred.reshape(y_pred.shape[0], 1, y_pred.shape[1])

pred_test_set = []
for index in range(0,len(y_pred)):
    pred_test_set.append(np.concatenate([y_pred[index], X_test[index]], axis=1))

pred_test_set = np.array(pred_test_set) 
pred_test_set = pred_test_set.reshape(pred_test_set.shape[0], pred_test_set.shape[2]) 
pred_test_set_inverted = scaler.inverse_transform(pred_test_set)

# Transformación inversa de las diferencias de ventas.
result_list = []
sales_dates = list(df[-13:].mes)
act_sales = list(df[-13:].ventas)
for index in range(0,len(pred_test_set_inverted)):
    result_dict = {}
    result_dict['mes'] = sales_dates[index+1]
    result_dict['pred_value'] = int(pred_test_set_inverted[index][0] + act_sales[index])
    result_dict['real_value'] = int(act_sales[index+1])
    result_list.append(result_dict)
df_result = pd.DataFrame(result_list)
df_result

12/12 [==============================] - 0s 2ms/step


,mes,pred_value,real_value
0,2018-08,258824898,190794276
1,2018-09,248764613,190349097
2,2018-10,231599215,188198737
3,2018-11,313008122,208732711
4,2018-12,170746666,147735452
5,2019-01,33855501,82625200
6,2019-02,200673320,127458810
7,2019-03,164998884,153793350
8,2019-04,154641575,138929019
9,2019-05,197442128,141128050


In [28]:
df_sales_pred = pd.merge(df,df_result,on='mes',how='left')
plot_data = [
go.Scatter(
x=df_sales_pred['mes'],
y=df_sales_pred['ventas'],
name='actual'
),
go.Scatter(
x=df_sales_pred['mes'],
y=df_sales_pred['pred_value'],
name='predicted'
)

]
plot_layout = go.Layout(
title='Predicción de ventas'
)
fig = go.Figure(data=plot_data, layout=plot_layout) 
pyoff.iplot(fig)

In [29]:
def next_month_predict(df, cantidad_meses):
    df_final = pd.DataFrame()
    for k in range(cantidad_meses):
        next_month = (datetime.strptime(str(list(df['mes'])[-1]), '%Y-%m') + relativedelta.relativedelta(months=1)).strftime('%Y-%m')
        df = df.append(pd.DataFrame(data={'mes': [next_month], 'ventas': [0]}))

        df_diff = df.copy()
        df_diff['prev_ventas'] = df_diff['ventas'].shift(1)
        df_diff = df_diff.dropna()
        df_diff['diff'] = (df_diff['ventas'] - df_diff['prev_ventas'])
        df_supervised = df_diff.drop(['prev_ventas'],axis=1)

        for inc in range(1,13):
            field_name = 'lag_' + str(inc)
            df_supervised[field_name] = df_supervised['diff'].shift(inc)

        df_supervised = df_supervised.dropna().reset_index(drop=True)
        df_model = df_supervised.drop(['ventas','mes'],axis=1)
        train_set, test_set = df_model[:-1].values, df_model[-1:].values

        scaler = MinMaxScaler(feature_range=(-1, 1))
        scaler = scaler.fit(train_set)

        train_set = train_set.reshape(train_set.shape[0], train_set.shape[1])
        train_set_scaled = scaler.transform(train_set)
        test_set = test_set.reshape(test_set.shape[0], test_set.shape[1])
        test_set_scaled = scaler.transform(test_set)

        X_train, y_train = train_set_scaled[:, 1:], train_set_scaled[:, 0:1]
        X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
        X_test, y_test = test_set_scaled[:, 1:], test_set_scaled[:, 0:1]
        X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

        model = Sequential()
        model.add(LSTM(4, batch_input_shape=(1, X_train.shape[1], X_train.shape[2]), stateful=True))
        model.add(Dense(1))
        model.compile(loss='mean_squared_error', optimizer='adam')
        model.fit(X_train, y_train, epochs=100, batch_size=1, verbose=1, shuffle=False)

        y_pred = model.predict(X_test,batch_size=1)
        y_pred = y_pred.reshape(y_pred.shape[0], 1, y_pred.shape[1])

        pred_test_set = []
        for index in range(0,len(y_pred)):
            pred_test_set.append(np.concatenate([y_pred[index], X_test[index]], axis=1))

        pred_test_set = np.array(pred_test_set)
        pred_test_set = pred_test_set.reshape(pred_test_set.shape[0], pred_test_set.shape[2])
        pred_test_set_inverted = scaler.inverse_transform(pred_test_set)

        result_list = []
        sales_dates = list(df[-2:].mes)
        act_sales = list(df[-2:].ventas)

        result_dic = {}
        result_dic['pred_value'] = int(act_sales[0])
        result_dic['mes'] = sales_dates[0]
        result_list.append(result_dic)

        for index in range(0,len(pred_test_set_inverted)):
            result_dict = {}
            result_dict['pred_value'] = int(pred_test_set_inverted[index][0] + act_sales[index])
            result_dict['mes'] = sales_dates[index+1]
            result_list.append(result_dict)
            
        df_result = pd.DataFrame(result_list)
        if k == 0:
            df_final['pred_value'] = list(df_result['pred_value'])
            df_final['mes'] = list(df_result['mes'])
        else:
            df_final = df_final.append(pd.DataFrame(data={'pred_value': [list(df_result['pred_value'])[-1]], 'mes': [list(df_result['mes'])[-1]]}))

        df = df[:-1]
        df = df.append(pd.DataFrame(data={'mes':[next_month], 'ventas':[list(df_result['pred_value'])[-1]]}))
        df = df.reset_index().drop('index', axis=1)
    

    # Graficamos los resultados.
    df_sales_pred = pd.merge(df,df_final,on='mes',how='left')
    plot_data = [
        go.Scatter(
            x=df_sales_pred['mes'][:cantidad_meses*-1],
            y=df_sales_pred['ventas'][:cantidad_meses*-1],
            name='actual'
        ),
            go.Scatter(
            x=df_sales_pred['mes'],
            y=df_sales_pred['pred_value'],
            name='predicted'
        )

    ]
    plot_layout = go.Layout(
            title='Predicción de Ventas'
        )
    fig = go.Figure(data=plot_data, layout=plot_layout)
    pyoff.iplot(fig)

In [30]:
next_month_predict(df, cantidad_meses=12)

Epoch 1/100
137/137 [==============================] - 2s 2ms/step - loss: 0.0579
Epoch 2/100
137/137 [==============================] - 0s 3ms/step - loss: 0.0504
Epoch 3/100
137/137 [==============================] - 0s 2ms/step - loss: 0.0436
Epoch 4/100
137/137 [==============================] - 0s 2ms/step - loss: 0.0374
Epoch 5/100
137/137 [==============================] - 0s 2ms/step - loss: 0.0324
Epoch 6/100
137/137 [==============================] - 0s 2ms/step - loss: 0.0285
Epoch 7/100
137/137 [==============================] - 0s 2ms/step - loss: 0.0256
Epoch 8/100
137/137 [==============================] - 0s 2ms/step - loss: 0.0234
Epoch 9/100
137/137 [==============================] - 0s 2ms/step - loss: 0.0218
Epoch 10/100
137/137 [==============================] - 0s 2ms/step - loss: 0.0206
Epoch 11/100
137/137 [==============================] - 0s 2ms/step - loss: 0.0196
Epoch 12/100
137/137 [==============================] - 0s 2ms/step - loss: 0.0189
Epoch 13/100
